## Introduction

<!--
This kernel shows how to use NBSVM (Naive Bayes - Support Vector Machine) to create a strong baseline for the [Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge) competition. NBSVM was introduced by Sida Wang and Chris Manning in the paper [Baselines and Bigrams: Simple, Good Sentiment and Topic Classiﬁcation](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf). In this kernel, we use sklearn's logistic regression, rather than SVM, although in practice the two are nearly identical (sklearn uses the liblinear library behind the scenes).

If you're not familiar with naive bayes and bag of words matrices, I've made a preview available of one of fast.ai's upcoming *Practical Machine Learning* course videos, which introduces this topic. Here is a link to the section of the video which discusses this: [Naive Bayes video](https://youtu.be/37sFIak42Sc?t=3745).
-->

이 노트북은 [Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge) 대회에서 좋은 베이스 라인을 만들기 위해 NBSVM(Naive Bayes - Support Vecotor Machine)를 사용하는 법을 보여준다. NBSVM은 Sida Wang과 Chris Manning이 [Baselines and Bigrams: Simple, Good Sentiment and Topic Classiﬁcation](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf) 논문에서 도입했다. 이 노트북에서, sklearn의 SVM 대신에 logistic regression 사용한다. 실제로, 이 둘은 거의 동일하지만 (sklearn은 liblinear 라이브러리를 사용한다.)

만약 여러분이 naive bayes와 bag of words matrices에 익숙하지 않는다면, fast.ai의 곧 올려질 *Practical Machine Learning* 코스 비디오에 중 하나를 미리보기로 만들었다. 거기서 이 주제에 대한 설명한다. 이것은 비디오에서 얘기할 섹션의 링크이다. [Naive Bayes video](https://youtu.be/37sFIak42Sc?t=3745).

In [ ]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')
subm = pd.read_csv('../input/sample_submission.csv.zip')

## Looking at the data

<!--
The training data contains a row per comment, with an id, the text of the comment, and 6 different labels that we'll try to predict.
-->

훈련 데이터는 댓글 당 id와 댓글의 텍스트로 이루어진 하나의 행으로 되어있다. 그리고 우리가 예측해야할 6개의 다른 label이 있다.

In [ ]:
train.head(10)

<!--
Here's a couple of examples of comments, one toxic, and one with no labels.
-->

toxic인 것들과 아닌 것들의 몇 가지의 댓글 예시가 있다.

In [ ]:
train['comment_text'][0]

In [ ]:
train['comment_text'][6]

<!--
The length of the comments varies a lot.
-->

댓글의 길이는 많이 다르다.

In [ ]:
lens = train.comment_text.str.len()
lens.mean(), lens.std(), lens.max()

In [ ]:
lens.hist();

<!--
We'll create a list of all the labels to predict, and we'll also create a 'none' label so we can see how many comments have no labels. We can then summarize the dataset.
-->

예측을 위한 모든 label 목록을 만들 것이다. 그리고 'none' 라벨도 만들 것이다. 그래서 얼마나 많은 댓글들에 label이 없는 지 확인할 수 있다. 그러면 데이터셋을 요약할 수도 있다.

In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

In [ ]:
len(train),len(test)

<!--
There are a few empty comments that we need to get rid of, otherwise sklearn will complain.
-->

제거해야할 몇 개의 비어있는 댓글이 있다. 제거를 안 한다면 sklearn이 작동하지 않을 것이다.

In [ ]:
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

## Building the model

<!--
We'll start by creating a *bag of words* representation, as a *term document matrix*. We'll use ngrams, as suggested in the NBSVM paper.
-->

*bag of words* 표현을 *term document matrix*로 만들면서 시작할 것이다. NBSVR 논문에서 제안한 ngram을 사용할 것이다.

In [ ]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

<!--
It turns out that using TF-IDF gives even better priors than the binarized features used in the paper. I don't think this has been mentioned in any paper before, but it improves leaderboard score from 0.59 to 0.55.
-->

TF-IDF를 사용하는 것이 논문에서 사용된 binarized features 보다 훨씬 더 좋다고 알려졌다. 이전에는 어떤 논문에서도 이것이 언급되었다고 생각하지 않는다. 그러나 이것이 leaderboard 점수를 0.59에서 0.55로 상승시켰다.

In [ ]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train[COMMENT])
test_term_doc = vec.transform(test[COMMENT])

<!--
This creates a *sparse matrix* with only a small number of non-zero elements (*stored elements* in the representation  below).
-->

이것은 0이 아닌 원소가 매우 적은 *sparse matrix* 를 만들어낸다. (아래 표현에서 *stored elements*)

In [ ]:
trn_term_doc, test_term_doc

<!--
Here's the basic naive bayes feature equation:
-->

기본적인 naive bayes feature 식:

In [ ]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [ ]:
x = trn_term_doc
test_x = test_term_doc

<!--
Fit a model for one dependent at a time:
-->

한 번에 하나의 의존 모델을 적용한다.

In [ ]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [ ]:
preds = np.zeros((len(test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

<!--
And finally, create the submission file.
-->

그리고 마지막으로, submission file을 만든다.

In [ ]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission.csv', index=False)